In [1]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import numpy as np
import requests as r

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
#ALL IS REMOVED

In [4]:
with open('./source/questionnaire.json') as json_file:
    data = json.load(json_file)
    data = data['top_container']

In [5]:
exclude = [43374942,43426483,43374941,43374933,43375020,43374922,43398762]

In [6]:
country_key = 43380772
funds = 43374826
contribution = 43374827

In [7]:
config = {
    "values":{
        "43374939":"Type of Action",
        "43374932":"Role Organisation",
        "43374862":"Responsible Actor",
        "43375025":"Reporting",
        "43374951":"Reporting and Evaluations",
        "43374934":"Outcome Evaluated",
        "43374949":"Geography",
        "43374915":"Source to Sea",
        "43374916":"Lifecycle of Plastics",
        "43374904":"Target Action",
        "43374931":"Impact",
        "43374917":"Pollutant Targeted",
        "43374905":"Sector",
        "43374920":"Funding",
        "43374943":"Duration",
    },
    "country":{
        "id": "43380772"
    }
}

In [8]:
all_childs = []

def getAllChilds(x, pid=None, p=None):
    qid = None
    text = None
    dependencies = None
    if 'q_no' in x:
        qid = str(x['q_no'])
    if p:
        if 'q_no' in p:
            qid = str(p['q_no'])
        if 'q_no' in x:
            qid = str(x['q_no'])
    if 'text' in x:
        text = x['text']
        if text == " ":
            text = p['text']
    if 'title' in x:
        text = x['title']
    if pid:
        pid = str(pid)
    if 'dependencies' in x:
        dependencies = json.loads(x['dependencies'])
        dependencies = dependencies[0]
        rule = int(dependencies['rule'])
        parent = [o for o in p['options'] if o['value'] == rule]
        #print("=====================")
        #print("Parent " + parent[0]['text'] + "| Children " + x['text'])
        #print("=====================")
        pid = str(parent[0]['id'])
    if text:
        text = BeautifulSoup(text)
        text = text.get_text()
        all_childs.append({
            "id": x['id'],
            "name": text.replace('\xa0',' ').replace('\n','').replace('  ',' '),
            "pid": pid,
            "qid": qid,
            "type": x["class"],
        })
    if 'options' in x:
        for z in x['options']:
            getAllChilds(z, x['id'], x)
    if 'children' in x:
        for z in x['children']:
            getAllChilds(z, x['id'], x)

In [9]:
for d in data['children']:
    getAllChilds(d)

In [10]:
df = pd.DataFrame(all_childs)

In [11]:
df = df[~df['type'].isin(['Page','Section','Note'])]

In [12]:
df['qpar'] = df['qid'].apply(lambda x: x.split('.')[0]).astype('int')

In [13]:
dfids = df['id'].to_list()

In [14]:
def replaceParentId(pid):
    if int(pid) not in dfids:
        return None
    return pid

In [15]:
df['pid'] = df['pid'].apply(replaceParentId) 

In [16]:
resp = pd.read_csv('./source/data-production_2020-08-03.csv')

In [17]:
q_rename = {}
for qs in list(resp):
    qid = qs.split(' ')[0]
    if qid[-1] == ".":
        qid = qid[:-1]
        qname = '. '.join(qs.split('. ')[1:])
        ids = df[(df['qid']==qid)]
        ids = ids[~(ids['type']=='SelectionOption')]
        try:
            if qname in list(ids['name'])[0]:
                qid = list(ids['id'])[0]
        except:
            pass
    q_rename.update({qs:qid})

In [18]:
records = resp.rename(columns=q_rename)

In [19]:
records = records.drop(exclude, axis=1)

In [20]:
records[country_key] = records[country_key].apply(lambda x: "Côte d'Ivoire" if "Cote d'Ivoire" in x else x)

In [21]:
pd.DataFrame(records[43374842].str.strip().unique(), columns=["organisation_names"]).to_csv('organisationlist.csv')

In [22]:
non_var = ['StartDate','CompletionDate','Unique', country_key]
results = []
for a in records.to_dict('records'):
    for b in a:
        if b not in non_var:
            opt = df[(df['pid'] == str(b)) & (df['type'] == 'SelectionOption')]
            values = []
            for c in opt.to_dict('records'):
                if c['name'] in str(a[b]):
                    values.append(c['id'])
            if len(values) == 0:
                values = np.nan
            if values == values:
                results.append({'uuid':a['Unique'],'qid':b,'values':values,'countries':a[country_key]})

In [23]:
res = pd.DataFrame(results)

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, query
import numpy as np
from app.models import Countries, Groups, Values, CountryGroups, Datapoints, DatapointCountries, DatapointValues, Questions, Answers
from app.connection import engine_url, write_data

In [25]:
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()

In [26]:
values = df.rename(columns={'id':'code','pid':'parent_code'})
values = values[['name','code','parent_code','type']]

In [27]:
values['code'] = values['code'].fillna("Empty").astype('str')
values['parent_code'] = values['parent_code'].fillna("Empty").astype('str')

In [28]:
def getgrandparent(x):
    if x['type'] == "SelectionOption":
        parent = values[values['code'] == str(x['parent_code'])]
        if parent.shape[0] > 0:
            parent = parent.to_dict('records')[0]
            if parent['type'] == "MultipleChoice" and parent['parent_code'] != "Empty":
                return str(parent['parent_code'])
            elif parent['type'] == "MultipleAnswer" and parent['parent_code'] != "Empty":
                return str(parent['parent_code'])
            else:
                return str(parent['code'])
    return str(x['parent_code'])

In [29]:
values['gp'] = values.apply(getgrandparent, axis=1)

In [30]:
def setDrop(x):
    if x['type'] in ["FreeText","GridRow","ScaleRow","Scale"]:
        return True
    if x['type'] == "MultipleChoice" and x['parent_code'] != "Empty":
        return True
    if x['type'] == "MultipleAnswer" and x['parent_code'] != "Empty":
        return True
    return False

In [31]:
values['drop'] = values.apply(setDrop, axis=1)

In [32]:
### FreeText Question (ftqs)
ftqs = values[values['type'] == 'FreeText']

In [33]:
values.drop(values[values['drop']].index,inplace=True)

In [34]:
def renameColumn(x):
    name = x['name']
    try:
        name = config['values'][str(x['code'])]
    except:
        pass
    return name

In [35]:
values['name'] = values.apply(lambda x: renameColumn(x), axis=1)
values = values.reset_index()
values['parent_code'] = values.apply(lambda x: x['gp'] if x['type'] == "SelectionOption" else x['parent_code'],axis=1)
values['id'] = values['code'].astype(int)

In [36]:
includes = []
for a in config['values']:
    includes.append(int(a))

In [37]:
values.drop(values[(~values['id'].isin(includes)) & (values['type'] != "SelectionOption")].index, inplace=True)

In [38]:
allparentids = values['code'].to_list()
allparentids.append("Empty")

In [39]:
values['drop'] = values['parent_code'].apply(lambda x: False if x in allparentids else True)
values.drop(values[values['drop']].index, inplace=True)

In [40]:
values['parent_code'] = values['parent_code'].apply(lambda x: x if x != "Empty" else None)

In [41]:
values = values[['name','code','parent_code','type']]

In [42]:
### Value Table
for value in values.to_dict('records'):
    run = True
    if value['type'] == 'SelectionList' and value['parent_code'] != None:
        run = False
    if run:
        value.update({'parent_id':None,'description':None})
        parent = session.query(Values).filter(Values.code == value['parent_code']).first()
        if parent:
            value.update({'parent_id':parent.id})
        input_data = Values(value)
        write_data(session, input_data, value, False)

In [43]:
### Questions Table
for item in ftqs.to_dict('records'):
    item.update({'parent_id':None, 'value_id': None})
    parent = False
    value = False
    if item['parent_code'] != 'Empty':
        parent = session.query(Questions).filter(Questions.code == item['parent_code']).first()
        value = session.query(Values).filter(Values.code == item['parent_code']).first()
    if parent:
        item.update({'parent_id':parent.id, 'type': item['type']})
    if value:
        item.update({'value_id':value.id, 'type': 'Dependency'})
    input_data = Questions(item)
    write_data(session, input_data, item, False)

In [44]:
### Datapoint Table
for uuid in set(list(records['Unique'])):
    rec = records.fillna(0)
    colval = rec[rec['Unique'] == uuid][[funds, contribution]]
    fund = int(colval.to_dict('records')[0][funds])
    contrib = int(colval.to_dict('records')[0][contribution])
    datapoint = {'uuid':uuid,'funds':fund,'contribution':contrib}
    input_data = Datapoints(datapoint)
    write_data(session, input_data, datapoint, False)

In [45]:
### DatapointCountries Table
for dc in records[['Unique', country_key]].to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == dc['Unique']).first()
    if dc[country_key] == dc[country_key]:
        for c in dc[country_key].split(','):
            country = session.query(Countries).filter(Countries.name == c).first()
            if country:
                datapoint_country = {
                    'datapoint_id':datapoint.id,
                    'country_id': country.id
                }
                input_data = DatapointCountries(datapoint_country)
                write_data(session, input_data, datapoint_country, False)

In [46]:
### DatapointValues Table
for values in res.to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == values['uuid']).first()
    for v in values['values']:
        value = session.query(Values).filter(Values.code == v).first()
        if value:
            datapoint_value = {
                'datapoint_id':datapoint.id,
                'value_id': value.id
            }
            input_data = DatapointValues(datapoint_value)
            write_data(session, input_data, datapoint_value, False)

In [47]:
### Answers Table
qids = []
for x in ftqs['code']:
    if int(x) in list(records.columns):
        qids.append(int(x))
        
for uuid in set(list(records['Unique'])):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == uuid).first()
    rec = records
    colval = rec[rec['Unique'] == uuid][qids]
    for code in list(colval.columns):
        question = session.query(Questions).filter(Questions.code == str(code)).first()
        value = colval[code].values[0]
        answer_value = {
            'datapoint_id': datapoint.id,
            'question_id': question.id,
            'value': [None if value != value else str(value)]
        }
        input_data = Answers(answer_value)
        write_data(session, input_data, answer_value, False)